In [10]:
import json
import glob, pickle
from collections import defaultdict

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima, FakeBelem

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder, random_clifford_circuit
from gnn import ExpValCircuitGraphModel
from qiskit.quantum_info import Clifford

In [9]:
backend1 = FakeLima()
backend2 = FakeBelem()
properties1 = get_backend_properties_v1(backend1)
properties2 = get_backend_properties_v1(backend2)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend1_noisy = AerSimulator.from_backend(backend1) # Noisy
backend2_noisy = AerSimulator.from_backend(backend2) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy1 = {'shots': 10000, 'backend': backend1_noisy, 'name': 'noisy'}
run_config_noisy2 = {'shots': 10000, 'backend': backend2_noisy, 'name': 'noisy'}

In [18]:
files = sorted(glob.glob('./data/circ_linked_to_pyg_data/*.txt'))
files

['./data/circ_linked_to_pyg_data/qasm_11_-0.1.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.2.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.3.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.4.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.5.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.6.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.7.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.8.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-0.9.txt',
 './data/circ_linked_to_pyg_data/qasm_11_-1.0.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.0.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.1.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.2.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.3.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.4.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.5.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.6.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.7.txt',
 './data/circ_linked_to_pyg_data/qasm_11_0.8.txt',
 './data/circ_linked_

In [ ]:
circs = defaultdict(list)

#for file in tqdm_notebook(files, leave=False):
for file in ['./data/circ_linked_to_pyg_data/qasm_11_-0.3.txt']:
    print(file)
    depth = int(file.split('_')[-2])
    if depth not in [3, 5, 7, 9, 11]:
        continue
    # exp_val_bin = float(file.split('_')[-1][:-4])

    # f_name = '_'.join([depth, round(exp_val_bin, 1)]) + '.json'
    f_name = file.split('/')[-1][5:-4] + '.json'
    with open('./data/pyg_data/'+f_name, 'r') as in_f:
        pyg_dicts = json.load(in_f)
        print('len(pyg_dicts)\t', len(pyg_dicts))

    qc_list = []
    with open(file, "r") as f:
        lines = f.read().split('-----')
        print('len(lines)\t', len(lines) - 1)
        print('exp_val, \t ideal_exp_value, \t noisy_exp_values')
        for i, line in enumerate(lines):
            splitted = line.split('\n')
            valid = splitted[3:]
            if len(valid) and valid[0].startswith('qasm'):
                valid = valid[1:]
            elif not len(valid):
                continue
            try:
                exp_val = float(splitted[1])
            except:
                exp_val = float(splitted[2])
            qasm_str = '\n'.join(valid)
            qc = QuantumCircuit.from_qasm_str(qasm_str)
            pyg_dict = pyg_dicts[i]
            print(exp_val, pyg_dict['ideal_exp_value'], pyg_dict['noisy_exp_values'])
            qc_list.append((exp_val, qc, pyg_dict))
    print('len(qc_list)', len(qc_list))
    circs[depth] += qc_list

In [106]:
# with open('./data/circ_parsed_pyg_data/parsed.pk', 'wb') as out:
#     pickle.dump(circs, out)

In [4]:
with open('./data/circ_parsed_pyg_data/parsed.pk', 'rb') as in_f:
    parsed = pickle.load(in_f)

In [9]:
len(parsed[3.0])

1976

In [14]:
parsed[5.0][9][1].draw(fold=-1)

░ ┌─┐            
q_0: ────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤M├────────────
       ┌─┴─┐            ┌───┐           ┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐┌────┐┌─────────┐┌──────────┐ ░ └╥┘      ┌─┐   
q_1: ──┤ X ├────■───────┤ X ├───────■───┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/4) ├┤ X ├┤ Rz(-π/4) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(-π/2) ├─░──╫───────┤M├───
       └───┘    │       └─┬─┘       │   └─────────┘└─────────┘└────┘└─────────┘└─────────┘└─┬─┘└──────────┘└─────────┘└────┘└─────────┘└──────────┘ ░  ║    ┌─┐└╥┘   
q_2: ───────────┼─────────┼─────────┼───────────────────────────────────────────────────────■───────────────────────────────────────────────────────░──╫────┤M├─╫────
              ┌─┴─┐       │       ┌─┴─┐                                                                                                             ░  ║ ┌─┐└╥┘ ║    
q_3: ─────────┤ X ├───────■───────┤ X ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────░──╫─┤M├─╫──╫────
     ┌───────┐├───┴┐┌────────────┐├───┴┐ ┌────────┐                                                                                                 ░  ║ └╥┘ ║  ║ ┌─┐
q_4: ┤ Rz(0) ├┤ √X ├┤ Rz(4.8112) ├┤ √X ├─┤ Rz(3π) ├─────────────────────────────────────────────────────────────────────────────────────────────────░──╫──╫──╫──╫─┤M├
     └───────┘└────┘└────────────┘└────┘ └────────┘                                                                                                 ░  ║  ║  ║  ║ └╥┘
c: 5/══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                                                                                                                       0  1  2  3  4

In [5]:
with open('./data/pyg_data/3_0.0.json', 'r') as in_f:
    pyg_d = json.load(in_f)

In [7]:
pyg_d[0]['circuit_graph']

{'nodes': {'DAGOpNode': [[1.5707963267948966,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    5.9698643286635694e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    9.355584184359312e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    0.026100000000000012,
    0.0,
    0.0,
    0.0,
    0.0,
    0,
    0],
   [0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    5.9698643286635694e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    9.355584184359312e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    0.026100000000000012,
    0.0,
    0.0,
    0.0,
    0.0,
    0.00019195510390342677,
    35.55555555555556],
   [6.80205519107807,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    5.9698643286635694e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    9.355584184359312e-05,
    0.0,
    0.0,
    0.0,
    0.0,
    0.026100000000000012,
    0.0,
    0.0,
    0.0,
    0.0,
    0,
    0],
   [

In [12]:
for depth, qc_list in tqdm_notebook(parsed.items()):
    circuits = list(list(zip(*qc_list))[1])
    print(int(depth), len(circuits))
    entries = []
    for circuit in tqdm_notebook(circuits[1400:1700]):
        job_ideal = execute(circuit, **run_config_ideal)
        job_noisy = execute(circuit, **run_config_noisy2)

        counts_ideal = job_ideal.result().get_counts()
        counts_noisy = job_noisy.result().get_counts()

        ideal_exp_val = cal_z_exp(counts_ideal)
        noisy_exp_val = cal_z_exp(counts_noisy)

        graph_data = circuit_to_graph_data_json(
            circuit=circuit,
            properties=properties2,
            use_qubit_features=True,
            use_gate_features=True,
        )

        entry = ExpValueEntry(
            circuit_graph=graph_data,
            observable=[],
            ideal_exp_value=ideal_exp_val.tolist(),
            noisy_exp_values=[noisy_exp_val.tolist()],
            circuit_depth=circuit.depth()
        )
        entries.append(entry.to_dict())

    print(len(entries))
    with open(f'./data/circ_parsed_pyg_data/finetune/fakebelem_depth{int(depth)}.json', 'w') as out_file:
        json.dump(entries, out_file)

  0%|          | 0/15 [00:00<?, ?it/s]

11 1953


  0%|          | 0/300 [00:00<?, ?it/s]

300
12 1984


  0%|          | 0/300 [00:00<?, ?it/s]

...

300
8 1978


  0%|          | 0/300 [00:00<?, ?it/s]

300
9 1971


  0%|          | 0/300 [00:00<?, ?it/s]

300
